<img src="./images/logo.png" alt="Drawing" style="width: 500px;"/>

# **Exercise 3:** Exploring Retail Data with Apache Spark

This exercise will introduce **Apache Spark on HPE AI Essentials**. We'll leverage Spark's powerful distributed processing capabilities to analyze and fix the sales information.

In this exercise, you will:

- Set up a Spark session for interacting with data.
- Generate sample sales data for different countries and currencies.
- Explore techniques for data loading, transformation, and analysis using Spark SQL and DataFrames.
- Create Delta Tables and perform version control.

Feel free to modify and extend the code examples to suit your specific data analysis needs.

Let's get started!

### **Prerequisites:**

As instructed in the [Introductory notebook](./00.introduction.ipynb), ensure that you have run `pip install -r requirements.txt` in a Terminal window, located in the same working directory, prior to running this notebook. 

<div class="alert alert-block alert-danger">
    <b>Important:</b> Make sure you selected <b>PySpark</b> for your notebook kernel - check the top right corner!
</div>

## **1. Create Spark Session**

Think about the most recent Excel spreadsheet you edited. It probably had tens or even hundreds of rows across tens of columns. When you run an Excel command, such as a *SUM()* or a *VLOOKUP()*, you may have noticed that it took a far bit of time to process. Maybe, even the fans of your laptop sped up a bit as your computer worked to crunch the numbers. 

Now, scale that same command out to a spreadsheet with tens of **millions** of rows across **thousands** of columns. That is the Big Data that companies must work with on a daily basis, and no single PC is going to run any *VLOOKUP* command on data of that size.

Instead of spreadsheets, the enterprise world is largely built upon **tables** in a variety of formats. To query these tables to retrieve certain data takes a **mammoth** amount of compute. It makes no sense to have a single **compute server** executing these queries - it would be far faster to parallelize queries across several computers. Enter **Apache Spark**.

### Introduction to Apache Spark on HPE AI Essentials

Apache Spark is a popular open-source big data framework that **distributes the computations** required to perform queries on large sets of data. This distribution, along with working with data in-memory rather than directly from storage disks, drastically brings down the time usually taken to query and index data. The combination of speed, versatility, and ease of use made Spark the go-to framework when working with big data. 

Apache Spark comes pre-installed with **HPE Ezmeral AI Essentials** and can leverage as much or as little of the compute available in a AIE cluster as a user desired. The core components of an Apache Spark deployment include:

<img src="./images/exercise1/spark_archi.PNG" alt="Drawing" style="width: 60%;"/>

**Driver:** The driver program coordinates the execution of Spark jobs. It submits tasks to executors, schedules operations, and manages communication between various components.

**Workers:** These are machines in the Spark cluster that manage executors. Each worker runs one or more executors. When running Spark on a HPE AI Essentials deployment, Spark Workers are Kubernetes pods distributed among worker nodes of the AIE cluster, allowing them to scale across multiple machines as required. 

**Executors:** Executors reside on worker nodes and carry out the actual computations assigned by the driver program. They partition and distribute the workload across machines in the cluster.

**JVM:**  Spark utilizes the Java Virtual Machine (JVM) on each worker node to execute executors.

On **HPE AI Essentials**, you will use Apache Spark to analyze large datasets at high speed with a unified platform for batch processing, streaming, and machine learning.

### Create a Spark Interactive Session

Let's begin using Spark! Here, you use HPE AI Essentials' native integration of **Apache Livy** to create and manage an interactive Spark session. Livy is an open-source REST service that enables remote and interactive analytics on Apache Spark clusters. It provides a way to interact with Spark clusters programmatically using a REST API, allowing you to submit Spark jobs, run interactive queries, and manage Sparksessions from web applications without the need for a specific Spark client. As a result, multiple AIE users can interact with your Spark cluster concurrently and reliably!

First, let's connect to the Livy endpoint and create a new Spark interactive session. The Spark interactive
session is particularly useful for exploratory data analysis, prototyping, and iterative development. It allows you to
interactively work with large datasets, perform transformations, apply analytical operations, and build ML models using
Spark's distributed computing capabilities. 

To communicate with Livy and manage your sessions you use Sparkmagic, an open-source tool that provides a Jupyter kernel
extension. Sparkmagic integrates with Livy, to provide the underlying communication layer between the Jupyter kernel and
the Spark cluster.

**Execute the cell below**, then:

1. Select the `Add Endpoint` tab.
1. Select `Single Sign-on` and ensure there is a Livy address in the `Address` field. 
1. Click `Add Endpoint`.
1. Select the `Create Session` tab.
1. Provide a name (e.g. `retail-demo`).
1. Select `python` under the Language field.
1. Click `Create Session` (right side).

The session will take a few minutes for your session to initialize. 

Once ready, the Manage Sessions pane will activate, displaying
your session ID. When the session state turns to idle, you're all set!

In [2]:
%manage_spark

MagicsControllerWidget(children=(Tab(children=(ManageSessionWidget(children=(HTML(value='<br/>'), HTML(value='…

Now, let's check the status of the session.

1. Navigate back to the AIE dashboard.
1. In the sidebar navigation menu, select `Spark Interactive Sessions`.

![image.png](./images/exercise1/menu.PNG)

3. Here, you can check the status of your session. It will take 2-3 minutes to start. When the `State` says `Idle`, the session is ready. 

![image.png](./images/exercise1/session.PNG)

4. Scroll back up to the Notebook cell of the session (%manage_spark command). Confirm under the `Manage Sessions` tab that the session should now be visible as `Idle` too. 

![image.png](./images/exercise1/session2.PNG)

### Configure Spark Interactive Session

1. Run the `%config_spark` magic command.
2. Leave the settings as they are. Click `Submit`.

<div class="alert alert-block alert-danger">
    <b>Important:</b> Ignore the resulting message and <b>do not</b> restart the kernel.
</div>

In [3]:
%config_spark

Button(description='Submit', style=ButtonStyle())

Output()

In [1]:
%sql

UsageError: Line magic function `%sql` not found (But cell magic `%%sql` exists, did you mean that instead?).


Next, let's import the required libraries for working with Spark in this notebook.

In [6]:
import os
os.makedirs("file:///mounts/shared-volume/shared/retail-data/delta-tables/vince", exist_ok=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
import random
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from delta.tables import DeltaTable

# Initialize Spark session
spark = SparkSession.builder \
    .appName("RetailDataPipeline") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Configuration
delta_path = "file:///mounts/shared-volume/shared/retail-data/delta-tables/"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
%%sql "SELECT * FROM retailvince.public.source_catalog"

UsageError: %%sql is a cell magic, but the cell body is empty.


In [33]:
presto_df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:presto://ezpresto.hpepcai.ezmeral.demo.local:/retailvince/public") \
    .option("dbtable", "source_catalog") \
    .option("driver", "com.facebook.presto.jdbc.PrestoDriver") \
    .load()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o195.load.
: java.sql.SQLException: No port number specified: jdbc:presto://ezpresto.hpepcai.ezmeral.demo.local/retailvince/public
	at com.facebook.presto.jdbc.PrestoDriverUri.parseDriverUrl(PrestoDriverUri.java:370)
	at com.facebook.presto.jdbc.PrestoDriverUri.<init>(PrestoDriverUri.java:114)
	at com.facebook.presto.jdbc.PrestoDriver.connect(PrestoDriver.java:86)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scala:160)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1$adapted(JdbcDialects.scala:156)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.getQueryOutputSchema(JDBCRDD.scala:63)
	at org

In [30]:
"""Extract data from SQL tables using %sql commands"""
print("Extracting data from SQL tables...")

# Extract each table and cache the DataFrames
catalog_df = spark.sql("SELECT * FROM retailvince.public.source_catalog")
customers_df = spark.sql("SELECT * FROM retailvince.public.source_customers")
orders_df = spark.sql("SELECT * FROM retailvince.public.source_orders")
order_products_df = spark.sql("SELECT * FROM retailvince.public.source_order_products")
stock_df = spark.sql("SELECT * FROM retailvince.public.source_stock")

return {
    "catalog": catalog_df,
    "customers": customers_df,
    "orders": orders_df,
    "order_products": order_products_df,
    "stock": stock_df
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
[REQUIRES_SINGLE_PART_NAMESPACE] spark_catalog requires a single-part namespace, but got `retailvince`.`public`.
Traceback (most recent call last):
  File "/opt/mapr/spark/spark-3.5.1/python/lib/pyspark.zip/pyspark/sql/session.py", line 1637, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery, litArgs), self)
  File "/opt/mapr/spark/spark-3.5.1/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/opt/mapr/spark/spark-3.5.1/python/lib/pyspark.zip/pyspark/errors/exceptions/captured.py", line 185, in deco
    raise converted from None
pyspark.errors.exceptions.captured.AnalysisException: [REQUIRES_SINGLE_PART_NAMESPACE] spark_catalog requires a single-part namespace, but got `retailvince`.`public`.



In [22]:
def create_delta_tables(dataframes):
    """Create Delta tables from the extracted data"""
    print("Creating Delta tables...")
    
    # Write each DataFrame to Delta format
    dataframes["catalog"].write.format("delta").mode("overwrite").save(f"{delta_path}source_catalog")
    dataframes["customers"].write.format("delta").mode("overwrite").save(f"{delta_path}source_customers")
    dataframes["orders"].write.format("delta").mode("overwrite").save(f"{delta_path}source_orders")
    dataframes["order_products"].write.format("delta").mode("overwrite").save(f"{delta_path}source_order_products")
    dataframes["stock"].write.format("delta").mode("overwrite").save(f"{delta_path}source_stock")
    
    print("Delta tables created successfully")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
def transform_data():
    """Transform the data in Delta tables"""
    print("Transforming data...")
    
    # Example transformation: Clean product names in catalog
    catalog_delta = DeltaTable.forPath(spark, f"{delta_path}source_catalog")
    catalog_df = catalog_delta.toDF()
    
    # Apply transformations
    cleaned_catalog = catalog_df.withColumn(
        "product_name_clean", 
        trim(col("product_name"))
    
    # Write transformed data back
    cleaned_catalog.write.format("delta").mode("overwrite").save(f"{delta_path}source_catalog")
    
    print("Data transformation completed")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
invalid syntax (<stdin>, line 15)
  File "<stdin>", line 15
    cleaned_catalog.write.format("delta").mode("overwrite").save(f"{delta_path}source_catalog")
    ^
SyntaxError: invalid syntax



In [ ]:
def validate_data():
    """Validate the data quality"""
    print("Validating data...")
    
    # Example validation: Check for null product names
    null_names = spark.sql(f"""
    SELECT COUNT(*) AS null_names_count 
    FROM delta.`{delta_path}source_catalog` 
    WHERE product_name IS NULL OR TRIM(product_name) = ''
    """).collect()[0]["null_names_count"]
    
    print(f"Found {null_names} products with null or empty names")
    
    # Add more validations as needed

In [ ]:
def main():
    """Main pipeline execution"""
    try:
        # Step 1: Extract data from SQL tables
        dataframes = extract_sql_tables()
        
        # Step 2: Create Delta tables
        create_delta_tables(dataframes)
        
        # Step 3: Transform data
        transform_data()
        
        # Step 4: Validate data
        validate_data()
        
        print("Pipeline executed successfully")
    except Exception as e:
        print(f"Pipeline failed: {str(e)}")
        raise

if __name__ == "__main__":
    main()

In [15]:
def create_delta_tables():
    """Create empty Delta tables with proper schema"""
    # Create products table
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS delta.`{delta_path}source_catalog` (
        product_id INT,
        product_name STRING,
        product_category STRING,
        price_cents INT
    ) USING DELTA
    """)
    
    # Create customers table
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS delta.`{delta_path}source_customers` (
        customer_id INT,
        customer_name STRING,
        customer_surname STRING,
        customer_email STRING
    ) USING DELTA
    """)
    
    # Create stock table
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS delta.`{delta_path}source_stock` (
        entry_id INT,
        product_id INT,
        product_quantity INT,
        purchase_price_cents INT,
        entry_date DATE
    ) USING DELTA
    """)
    
    # Create orders table
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS delta.`{delta_path}source_orders` (
        order_id INT,
        customer_id INT,
        order_status STRING,
        order_date DATE
    ) USING DELTA
    """)
    
    # Create order products table
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS delta.`{delta_path}source_order_products` (
        transaction_id INT,
        order_id INT,
        product_id INT,
        product_quantity INT
    ) USING DELTA
    """)
    
    print("Delta tables created successfully")

def add_constraints():
    """Add constraints and metadata after table creation"""
    # For Delta Lake, we can add constraints as table properties or enforce them through application logic
    # This is a placeholder for where you would implement your constraint logic
    print("Constraints would be enforced here through application logic")

    # Example: You could create a function that validates data before writing
    # or use Delta Lake's data validation features

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
def load_raw_data_to_delta():
    """Load generated sample data to Delta tables"""
    data = generate_sample_data()
    
    # Convert each dataset to Spark DataFrame and write to Delta
    spark.createDataFrame(data["products"]).write \
        .format("delta") \
        .mode("append") \
        .save(f"{delta_path}source_catalog")
    
    spark.createDataFrame(data["customers"]).write \
        .format("delta") \
        .mode("append") \
        .save(f"{delta_path}source_customers")
    
    # Similarly for other tables...
    print("Sample data loaded to Delta tables")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
def transform_data():
    """Apply transformations to the raw data"""
    # Example transformation: Clean product names
    products_df = DeltaTable.forPath(spark, f"{delta_path}source_catalog").toDF()
    
    cleaned_products = products_df.withColumn(
        "product_name_clean",
        col("product_name").rtrim().ltrim()
    ).drop("product_name").withColumnRenamed("product_name_clean", "product_name")
    
    # Example transformation: Standardize categories
    category_mapping = {
        "Toyz": "Toys",
        "Clothng": "Clothing",
        "Eletronics": "Electronics"
    }
    
    mapping_expr = "CASE "
    for wrong, correct in category_mapping.items():
        mapping_expr += f"WHEN product_category = '{wrong}' THEN '{correct}' "
    mapping_expr += "ELSE product_category END"
    
    cleaned_products = cleaned_products.withColumn(
        "product_category_clean",
        expr(mapping_expr)
    ).drop("product_category").withColumnRenamed("product_category_clean", "product_category")
    
    # Save transformed data to new Delta table
    cleaned_products.write \
        .format("delta") \
        .mode("overwrite") \
        .save(f"{delta_path}transformed_catalog")
    
    print("Data transformations completed")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
def validate_data():
    """Run data quality checks"""
    # Example validation: Check for null product names
    null_names = spark.sql(f"""
    SELECT COUNT(*) AS null_names_count 
    FROM delta.`{delta_path}source_catalog` 
    WHERE product_name IS NULL OR TRIM(product_name) = ''
    """).collect()[0]["null_names_count"]
    
    print(f"Found {null_names} products with null or empty names")
    
    # Add more validations as needed

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
"""Orchestrate the data pipeline"""
try:
    # Step 1: Create Delta tables
    create_delta_tables()
    
    # Step 2: Load raw data
    load_raw_data_to_delta()
    
    # Step 3: Validate raw data
    validate_data()
    
    # Step 4: Transform data
    transform_data()
    
    # Step 5: Validate transformed data
    validate_data()
    
    print("Pipeline executed successfully")
except Exception as e:
    print(f"Pipeline failed: {str(e)}")
    raise

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient
Traceback (most recent call last):
  File "<stdin>", line 4, in create_delta_tables
  File "/opt/mapr/spark/spark-3.5.1/python/lib/pyspark.zip/pyspark/sql/session.py", line 1637, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery, litArgs), self)
  File "/opt/mapr/spark/spark-3.5.1/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/opt/mapr/spark/spark-3.5.1/python/lib/pyspark.zip/pyspark/errors/exceptions/captured.py", line 185, in deco
    raise converted from None
pyspark.errors.exceptions.captured.AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient



In [20]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.functions import udf, col, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
import os

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We will also define the paths for where Spark will pull files from and save files to. These paths are specific to the AIE directory structure and to be left as they are.

In [8]:
file_root = "file:///mounts/shared-volume/shared/retail-data/raw-data"
delta_root = "file:///mounts/shared-volume/shared/retail-data/delta-tables/"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

You can now instantiate the Spark session. We'll add delta extensions to the configuration to be able to interact with the delta tables.

In [9]:
# Set up the Spark session
spark = SparkSession.builder \
    .appName("DataCleaningWithSpark") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.local.dir", "/mnt/shared/end2end-main-exercises/exercises") \
    .getOrCreate()

print("Pyspark session started")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Pyspark session started

## **2. Generating and Preparing Sales Data**

In this section, we are going to synthetically generate several years of sales data from our three retail stores located in three countries: Switzerland, Germany and the Czech Republic. This sales data will provide the basis for the remaining exercises, where we will learn to analyze, graph and build dashboards to gather insights between and across regions. 

**Optional:** To use `Data Sources` connected through AIE (such as MySQL, MariaDB and PostgresSQL databases), follow **this**.


### Generating Sales Data

A Python script has been provided which can generate the sales data for the three given country locations. 

The parameters for this script are:

- cu: Currency, to account for conversions between stores.
- s: Number of stores in that region.
- sy: Start Year
- ey: End Year
- csv: Resulting File Name

We'll see the first 10 rows of the newly created table. 

In [1]:
%run resources/create_csv.py -c "Germany" -cu EUR -s 5 -sy 2019 -ey 2023 -csv "germany_sales_data_2019_2023.csv" 

/home/admin/.local/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning:

Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning



# Variables Selected
	- Country: Germany
	- Currency: EUR
	- Stores: 5
	 ['Store A', 'Store B', 'Store C', 'Store D', 'Store E']
	- Start: 2019
	- End: 2023
# File /mnt/shared/retail-data/raw-data/germany_sales_data_2019_2023.csv created
['PRODUCTID', 'PRODUCT', 'TYPE', 'UNITPRICE', 'UNIT', 'QTY', 'TOTALSALES', 'CURRENCY', 'STORE', 'COUNTRY', 'YEAR']
['1', 'banana', 'fruits', '0.5', 'kg', '840256', '420128.0', 'EUR', 'Store A', 'Germany', '2019']
['2', 'apple', 'fruits', '0.7', 'kg', '811684', '988306.8', 'EUR', 'Store A', 'Germany', '2019']
['3', 'pear', 'fruits', '0.6', 'kg', '823589', '1482460.2', 'EUR', 'Store A', 'Germany', '2019']
['4', 'grapes', 'fruits', '1.2', 'kg', '411794', '1976613.0', 'EUR', 'Store A', 'Germany', '2019']
['5', 'orange', 'fruits', '0.8', 'kg', '802756', '2618817.8', 'EUR', 'Store A', 'Germany', '2019']
['6', 'kiwi', 'fruits', '0.9', 'piece', '795811', '3335047.7', 'EUR', 'Store A', 'Germany', '2019']
['7', 'watermelon', 'fruits', '5.0', 'kg', '102532', '384

In [2]:
%run resources/create_csv.py -c "Czech Republic" -cu CZK -s 5 -sy 2019 -ey 2023 -csv "czech_sales_data_2019_2023.csv"

# Variables Selected
	- Country: Czech Republic
	- Currency: CZK
	- Stores: 5
	 ['Store A', 'Store B', 'Store C', 'Store D', 'Store E']
	- Start: 2019
	- End: 2023
# File /mnt/shared/retail-data/raw-data/czech_sales_data_2019_2023.csv created
['PRODUCTID', 'PRODUCT', 'TYPE', 'UNITPRICE', 'UNIT', 'QTY', 'TOTALSALES', 'CURRENCY', 'STORE', 'COUNTRY', 'YEAR']
['1', 'banana', 'fruits', '12.52', 'kg', '50259', '629242.68', 'CZK', 'Store A', 'Czech Republic', '2019']
['2', 'apple', 'fruits', '17.528', 'kg', '25985', '1084707.76', 'CZK', 'Store A', 'Czech Republic', '2019']
['3', 'pear', 'fruits', '15.024', 'kg', '26461', '1482257.82', 'CZK', 'Store A', 'Czech Republic', '2019']
['4', 'grapes', 'fruits', '30.048', 'kg', '13230', '1879792.86', 'CZK', 'Store A', 'Czech Republic', '2019']
['5', 'orange', 'fruits', '20.032', 'kg', '25629', '2393192.99', 'CZK', 'Store A', 'Czech Republic', '2019']
['6', 'kiwi', 'fruits', '22.536', 'piece', '25351', '2964503.13', 'CZK', 'Store A', 'Czech Republic', 

In [3]:
%run resources/create_csv.py -c "Swiss" -cu CHF -s 5 -sy 2019 -ey 2023 -csv "swiss_sales_data_2019_2023.csv"

# Variables Selected
	- Country: Swiss
	- Currency: CHF
	- Stores: 5
	 ['Store A', 'Store B', 'Store C', 'Store D', 'Store E']
	- Start: 2019
	- End: 2023
# File /mnt/shared/retail-data/raw-data/swiss_sales_data_2019_2023.csv created
['PRODUCTID', 'PRODUCT', 'TYPE', 'UNITPRICE', 'UNIT', 'QTY', 'TOTALSALES', 'CURRENCY', 'STORE', 'COUNTRY', 'YEAR']
['1', 'banana', 'fruits', '0.4775', 'kg', '844968', '403472.22', 'CHF', 'Store A', 'Swiss', '2019']
['2', 'apple', 'fruits', '0.6685', 'kg', '815050', '948333.14', 'CHF', 'Store A', 'Swiss', '2019']
['3', 'pear', 'fruits', '0.573', 'kg', '827516', '1422499.81', 'CHF', 'Store A', 'Swiss', '2019']
['4', 'grapes', 'fruits', '1.146', 'kg', '413758', '1896666.48', 'CHF', 'Store A', 'Swiss', '2019']
['5', 'orange', 'fruits', '0.764', 'kg', '805701', '2512222.04', 'CHF', 'Store A', 'Swiss', '2019']
['6', 'kiwi', 'fruits', '0.8594999999999999', 'piece', '798429', '3198471.77', 'CHF', 'Store A', 'Swiss', '2019']
['7', 'watermelon', 'fruits', '4.7749999

Next, we'll ensure that our Spark Interactive session can access the data.

In [10]:
# Define the directory path
data_path = file_root

# List files in the directory
files = spark.sparkContext.wholeTextFiles(data_path)

# Display the list of files
for file_path, _ in files.collect():
    print(file_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

file:///mounts/shared-volume/shared/retail-data/raw-data/czech_sales_data_2019_2023.csv
file:///mounts/shared-volume/shared/retail-data/raw-data/germany_sales_data_2019_2023.csv
file:///mounts/shared-volume/shared/retail-data/raw-data/swiss_sales_data_2019_2023.csv

## **3. Create Delta Tables**

In this section, we will create Delta Tables from our CSV files that we can query using AIE. Delta Tables are a type of table that can be created in Delta Lake, which is an extension of Apache Parquet file format.

### Define an ETL Pipeline to create Delta Tables 

First, let's define some functions that will:

1. Load the data in from a CSV and return a pandas DataFrame.

In [11]:
from pyspark.sql.types import IntegerType

def load_data(spark, country, data_path):
    # Define the path to the CSV file
    csv_path = f"{data_path}/{country}_sales_data_2019_2023.csv"

    # Define the schema with specific data types
    schema = StructType([
        StructField("PRODUCTID", IntegerType(), True),
        StructField("PRODUCT", StringType(), True),
        StructField("TYPE", StringType(), True),
        StructField("UNITPRICE", DoubleType(), True),
        StructField("UNIT", StringType(), True),
        StructField("QTY", IntegerType(), True),
        StructField("TOTALSALES", DoubleType(), True),
        StructField("CURRENCY", StringType(), True),
        StructField("STORE", StringType(), True),
        StructField("COUNTRY", StringType(), True),
        StructField("YEAR", IntegerType(), True)
    ])

    # Read data from the CSV file with the specified schema
    df = spark.read \
        .format("csv") \
        .schema(schema) \
        .option("header", "true") \
        .load(csv_path)

    return df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2. Clean the data, in this case by ensuring the currency of each item is standardized in Euros.

In [12]:
def clean_data(df, spark, country):
    # Define a UDF to convert currencies to EUR
    convert_udf = udf(lambda currency, amount: amount / CZK_TO_EUR_RATE if currency == "CZK" else amount / CHF_TO_EUR_RATE if currency == "CHF" else amount, DoubleType())

    # Apply the UDFs to the DataFrame
    corrected_df = df.withColumn("totalsales", convert_udf(col("currency"), col("totalsales"))) \
                     .withColumn("currency", lit("EUR"))

    # Show the results
    corrected_df.show()

    return corrected_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3. Save the data as parquet files (Delta Tables).

In [13]:
def write_data(df, country):
    delta_path = delta_root + country

    # Check if the directory exists, and create it if it doesn't
    if not os.path.exists(delta_path):
        os.makedirs(delta_path)
        
    df.write.format("delta").mode("overwrite").save(delta_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Great! We've just created functions that will **extract** the data from our generated CSV files, **transform** them into Delta Tables with the currency standardized, then **load** them into a new directory.

You guessed it! We have just created an **ETL pipeline!** 

After declaring our country list and our currency conversion rates, we can run the pipeline.

In [14]:
# Constants
COUNTRY_LIST = ["czech", "germany", "swiss"]
CZK_TO_EUR_RATE = 25
CHF_TO_EUR_RATE = 1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-warning">
<b>Hint:</b> As you can tell by the parameters to the create_csv.py functions in Section 2, we can synthetically generate data for as many stores in as many European countries as we want! Feel free to experiment, so long as the countries are declared in the cell above <b>and the countries that are already there remain.</b>
</div>

In [15]:
for country in COUNTRY_LIST:
    # Load data from the DBs
    df = load_data(spark, country, data_path)
    df.show()
    
    # Clean the data
    cleaned_df = clean_data(df, spark, country)
    cleaned_df.printSchema()
    
    # Write the cleaned data back to the Delta Table
    write_data(cleaned_df, country)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----------+----------+------------------+-----+------+----------+--------+-------+--------------+----+
|PRODUCTID|    PRODUCT|      TYPE|         UNITPRICE| UNIT|   QTY|TOTALSALES|CURRENCY|  STORE|       COUNTRY|YEAR|
+---------+-----------+----------+------------------+-----+------+----------+--------+-------+--------------+----+
|        1|     banana|    fruits|             12.52|   kg| 50259| 629242.68|     CZK|Store A|Czech Republic|2019|
|        2|      apple|    fruits|            17.528|   kg| 25985|1084707.76|     CZK|Store A|Czech Republic|2019|
|        3|       pear|    fruits|            15.024|   kg| 26461|1482257.82|     CZK|Store A|Czech Republic|2019|
|        4|     grapes|    fruits|            30.048|   kg| 13230|1879792.86|     CZK|Store A|Czech Republic|2019|
|        5|     orange|    fruits|            20.032|   kg| 25629|2393192.99|     CZK|Store A|Czech Republic|2019|
|        6|       kiwi|    fruits|            22.536|piece| 25351|2964503.13|   

Now, we'll confirm the Delta Tables were create correctly.

In [16]:
for country in COUNTRY_LIST:
    # List files in a directory
    selected_country_path = delta_root + country
    files = os.listdir(selected_country_path)
    print("Table:", country)
    
    for file in files:
        if file.endswith(".parquet"):
            full_path = os.path.join(selected_country_path, file)
            print("Saved in:", full_path)

    print()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Table: czech

Table: germany

Table: swiss

# **Conclusion**

In this exercise, you learned to perform the basics of data engineering - all within a single notebook! 

**HPE AI Essentials** makes this possible by natively supporting and including the most widely used open-source data tools and frameworks and making them readily available out-of-the-box, such that you spent this time performing invaluable data preperation for upcoming exercises instead of hours installing and connecting them all!

In the next exercise, you will learn how to use EzPresto on HPE AI Essentials to prepare these datasets for visualization and modelling. 

